# The Stanford Sentiment Treebank 
The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [3]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export BUCKET_TRANSLATION_NAME=your_gcp_gs_bucket_translation_name`  
`export BUCKET_STAGING_NAME=your_gcp_gs_bucket_staging_name` 
`export REGION=your_region`  
`export PATH_SAVE_MODEL=your_path_to_save_model`  
`export CLOUDSDK_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  
`export CLOUDSDK_GSUTIL_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    # [EDIT ME]
    export DIR_PROJ=your_path_git_repository
    export PYTHONPATH=$DIR_PROJ/src
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1='\[\e[91m\]\u@:\[\e[32m\]\w\[\e[0m\]$'
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import Packages

In [4]:
import tensorflow as tf
import tensorflow_datasets

from tensorflow.keras.utils import to_categorical

from transformers import (
    BertConfig,
    BertTokenizer,
    XLMRobertaTokenizer,
    TFBertModel,
    TFXLMRobertaModel,
    TFBertForSequenceClassification,
    glue_convert_examples_to_features,
    glue_processors
)

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

from google.cloud import storage

import math
import numpy as np
import os
import glob
import time
from datetime import timedelta
import shutil
from datetime import datetime
import pickle
import re
import codecs
import json

from google.api_core.client_options import ClientOptions
from googleapiclient import discovery
from googleapiclient import errors

## Check configuration

In [5]:
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.3.0-rc2-23-gb36436b087 2.3.0


In [6]:
print(tf.keras.__version__)

2.4.0


In [7]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)>0:
    for gpu in gpus:
        print('Name:', gpu.name, '  Type:', gpu.device_type)
else:
    print('No GPU available !!!!')

No GPU available !!!!


## Define Paths

In [8]:
try:
    data_dir=os.environ['PATH_DATASETS']
except KeyError:
    print('missing PATH_DATASETS')
try:   
    tensorboard_dir=os.environ['PATH_TENSORBOARD']
except KeyError:
    print('missing PATH_TENSORBOARD')
try:   
    savemodel_dir=os.environ['PATH_SAVE_MODEL']
except KeyError:
    print('missing PATH_SAVE_MODEL')

## Import local packages

In [9]:
import preprocessing.preprocessing as pp
import utils.model_metrics as mm
import utils.model_utils as mu

In [10]:
import importlib
importlib.reload(pp);
importlib.reload(mm);
importlib.reload(mu);

## Check the census model stored on GCP

The **variables** directory contains a standard training checkpoint (see the guide to training checkpoints).  
The **assets** directory contains files used by the TensorFlow graph, for example text files used to initialize vocabulary tables.  
The **saved_model.pb** file stores the actual TensorFlow program, or model, and a set of named signatures, each identifying a function that accepts tensor inputs and produces tensor outputs.

In [12]:
# use model trainied with CPU
os.environ['MODEL_GCP']='gs://'+os.environ['BUCKET_NAME']+'/census_20200624_101711/keras-job-dir/4/keras_export'

In [13]:
%%bash
saved_model_cli show --dir $MODEL_GCP --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 11)
      name: serving_default_dense_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_4'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


## Model serving setup

In [14]:
# Normal VM has a model size of 500 MB 
# For more you need to use a specific n1-standard-2 VM (2 GB) for online prediction. It is only available in us-central1.

region_model = 'us-central1'
#region_model = 'europe-west4'
#region_model = 'europe-west1'
#region_model = 'europe-west6'

In [15]:
regional_endpoint=False
if region_model=='europe-west4':
    regional_endpoint=True
elif region_model=='us-central1':
    regional_endpoint=True
print(' Region: {} is a regional endpoint: {}'.format(region_model, regional_endpoint))

 Region: us-central1 is a regional endpoint: True


In [16]:
batch_pred=True
if batch_pred:
    regional_endpoint=False

In [17]:
project_name = os.environ['PROJECT_ID']
project_id = 'projects/{}'.format(project_name)
if not regional_endpoint:
    ai_platform_serving = discovery.build('ml', 'v1')
else:
    endpoint = 'https://'+region_model+'-ml.googleapis.com'
    client_options = ClientOptions(api_endpoint=endpoint)
    ai_platform_serving = discovery.build('ml', 'v1', client_options=client_options)
# to list all model
ai_platform_serving_global = discovery.build('ml', 'v1')

/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is a

### Check models already deployed

In [18]:
request = ai_platform_serving_global.projects().models().list(parent=project_id)

In [19]:
# Make the call.
try:
    response = request.execute()
    print('List of model:')
    if 'models' in response.keys():
        for i in response['models']:
            print('  Model \'s name: {}:'.format(i['name'].split('/')[-1]))
            print('    descrition: {}'.format(i['description']))
            print('    regions: {}'.format(i['regions']))

except errors.HttpError as err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

List of model:


### Create a new model

In [25]:
# defining the name of the model for online prediction
if batch_pred:
    name_model = 'tf_gcp_census_test_batch_'+region_model.replace('-','_')
else:
    name_model = 'tf_gcp_census_test_'+region_model.replace('-','_')
description_model = 'this is a model for test using census gcp code'

# Create a dictionary with the fields from the request body.
request_dict = {'name': name_model,
                'regions': [region_model],
                'description': description_model,
                'labels': {'region': region_model} }

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().models().create(parent=project_id, 
                                                         body=request_dict)

In [26]:
request_dict 

{'name': 'tf_gcp_census_test_batch_us_central1',
 'regions': ['us-central1'],
 'description': 'this is a model for test using census gcp code',
 'labels': {'region': 'us-central1'}}

In [27]:
# Make the call.
try:
    response = request.execute()
    print('Name of the model: {}:'.format(response['name'].split('/')[-1]))
    print('  descrition: {}'.format(response['description']))
    print('  regions: {}'.format(response['regions']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

Name of the model: tf_gcp_census_test_batch_us_central1:
  descrition: this is a model for test using census gcp code
  regions: ['us-central1']


### Defined all parameters and upload our models

In [33]:
# defining the name of the model for online prediction
parentId = 'projects/{}/models/{}'.format(project_name, name_model)
# Normal VM has a model size of 500 MB for more you need to use a specific n1-standard-2 VM (2 GB) for online prediction. It is only available in us-central1.
#region_model = 'us-central1'

model_binaries = os.environ['MODEL_GCP'] 
machine_type='mls1-c1-m2'
version = 'V1'

# Create a dictionary with the fields from the request body.
request_dict = {'machineType': machine_type,
                'runtimeVersion': '2.1',
                'pythonVersion': '3.7',
                'framework': 'TENSORFLOW',
                'description': description_model,
                'deploymentUri': model_binaries,
                'name': version
               }

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().models().versions().create(parent=parentId, 
                                                                    body=request_dict)

In [64]:
#request_dict

In [35]:
# Make the call.
try:
    response = request.execute()
    print('Name of the model: {}:'.format(response['name'].split('/')[-1]))
    print('  descrition: {}'.format(response['metadata']['version']['description']))
    print('  runtimeVersion: {}'.format(response['metadata']['version']['runtimeVersion']))
    print('  framework: {}'.format(response['metadata']['version']['framework']))
    print('  machineType: {}'.format(response['metadata']['version']['machineType']))
    print('  pythonVersion: {}'.format(response['metadata']['version']['pythonVersion']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

Name of the model: create_tf_gcp_census_test_batch_us_central1_V1-1598361592671:
  descrition: this is a model for test using census gcp code
  runtimeVersion: 2.1
  framework: TENSORFLOW
  machineType: mls1-c1-m2
  pythonVersion: 3.7


### Check that the new modelal was deployed

In [36]:
request = ai_platform_serving.projects().models().list(parent=project_id)

In [37]:
# Make the call.
try:
    response = request.execute()
    print('List of model:')
    for i in response['models']:
        print('  Model \'s name: {}:'.format(i['name'].split('/')[-1]))
        print('    descrition: {}'.format(i['description']))
        print('    regions: {}'.format(i['regions']))

except errors.HttpError as err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

List of model:
  Model 's name: tf_gcp_census_test_batch_us_central1:
    descrition: this is a model for test using census gcp code
    regions: ['us-central1']


## Model serving inference

### Prepare data for online prediction for BERT

example of format:

```
{'instances': 
  [
    {'input_ids': [101, 143, 18267, 15470, 90395, ...], 
     'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, .....], 
     'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, .....]
     }, 
     {'input_ids': [101, 17664, 143, 30728, .........], 
      'attention_mask': [1, 1, 1, 1, 1, 1, 1, .......], 
      'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, ....]
      }
  ]
}

```

### Prepare data for online prediction for Census

In [38]:
# for census
data_prediction=[{'dense_input': [25.0, 0, 7, 0, 0, 0, 0, 0, 0, 40, 0]}]

In [39]:
data_prediction[0]

{'dense_input': [25.0, 0, 7, 0, 0, 0, 0, 0, 0, 40, 0]}

### Online prediction

In [40]:
# model
version_name='V1'

# use the one above or define it below
#name_model='...'

# use the one above or define it below, be careful with regional endpoint
#ai_platform_serving='...'

parent = 'projects/{}/models/{}/versions/{}'.format(project_name, name_model, version_name)

# data for prediction
request_data = {"instances": data_prediction}

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().predict(body=request_data, 
                                                 name=parent)

In [44]:
#parent

In [65]:
#request_data

In [43]:
# Make the call.
try:
    response = request.execute()
    print('predictions:')
    for i in response['predictions']:
        print('  {}'.format(i))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error making prediction. Check the details:')
    print(err._get_reason())

predictions:
  {'dense_4': [0.2507100999355316]}


### Batch predictions

In [48]:
# Regional endpoint only support online prediction and AI explanations
#ai_platform_serving = discovery.build('ml', 'v1')
#name_model='...'
version_name=None
input_paths='gs://'+os.environ['BUCKET_NAME']+'/serving/sst2/input_predict_gcloud_census.json'
output_path='gs://'+os.environ['BUCKET_NAME']+'/batch_prediction_census_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
data_format='TEXT'
max_worker_count=20
runtime_version=None

not_deployed=False
if not_deployed:
    runtime_version='2.1'
    uri='gs://'+os.environ['BUCKET_NAME']+'/census_20200706_194610/keras-job-dir/4/keras_export'
    signatureName='serving_default'

In [60]:
model_id = '{}/models/{}'.format(project_id, name_model)
if version_name is not None:
    version_id = '{}/versions/{}'.format(model_id, version_name)

# Make a jobName of the format "model_name_batch_predict_YYYYMMDD_HHMMSS"
timestamp = time.strftime('%Y%m%d_%H%M%S', time.gmtime())

job_id = '{}_{}'.format(name_model,timestamp)
# Start building the request dictionary with required information.
body = {'jobId': job_id,
        'predictionInput': {
            'dataFormat': data_format,
            'inputPaths': [input_paths],
            'outputPath': output_path,
            'region': region_model}
       }

# Use the version if present, the model (its default version) if not.
if not_deployed:
    body['predictionInput']['uri'] = uri
    body['predictionInput']['signatureName'] = signatureName
else:
    if version_name is not None:
        body['predictionInput']['versionName'] = version_id
    else:
        body['predictionInput']['modelName'] = model_id

# Only include a maximum number of workers or a runtime version if specified.
# Otherwise let the service use its defaults.
#if max_worker_count:
#    body['predictionInput']['maxWorkerCount'] = max_worker_count

if runtime_version:
    body['predictionInput']['runtimeVersion'] = runtime_version

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().jobs().create(parent=project_id,
                                                       body=body)

In [61]:
body

{'jobId': 'tf_gcp_census_test_batch_us_central1_20200825_134940',
 'predictionInput': {'dataFormat': 'TEXT',
  'inputPaths': ['gs://multilingual_text_classification/serving/sst2/input_predict_gcloud_census.json'],
  'outputPath': 'gs://multilingual_text_classification/batch_prediction_census_2020_08_25_153129',
  'region': 'us-central1',
  'modelName': 'projects/axarevvicnonprod/models/tf_gcp_census_test_batch_us_central1'}}

In [63]:
# Make the call.
try:
    response = request.execute()
    print('job requested.')

    # The state returned will almost always be QUEUED.
    print('state : {}'.format(response['state']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error making prediction. Check the details:')
    print(err._get_reason())

job requested.
state : QUEUED


## Test with gcloud

In [57]:
# gcloud command test: OK
#!gcloud ai-platform jobs submit prediction 'tf_bert_classification_test_batch_us_central1_20200825_test_v1' \
#    --model 'tf_gcp_census_test_batch_us_central1' \
#    --input-paths 'gs://multilingual_text_classification/serving/sst2/input_predict_gcloud_census.json' \
#    --output-path 'gs://multilingual_text_classification/test_v1' \
#    --region 'us-central1' \
#    --data-format 'TEXT'

In [58]:
#!gcloud ai-platform jobs describe job_name...

In [59]:
#!gcloud ai-platform jobs stream-logs job_name...